In [5]:
import tensorflow as tf
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import mediapipe as mp

In [62]:
class GuitarChordDetectModel:
    def __init__(self, data_path: str):
        self.data_path = data_path
        self.file_names = os.listdir(data_path)
        self.output_images_path="data\\frame_images"

        for i in range(len(self.file_names)):
            self.file_names[i] = os.path.join(self.data_path, self.file_names[i])
        print(self.file_names)
        print(f"{len(self.file_names)} files")

    def play_video_file(self, video_file=None, save_frames=False, frame_save_interval=0.2):
        # Set the first video as default video
        if video_file is None:
            video_file = self.file_names[0]

        print(f"video file: {video_file}")

        cap = cv2.VideoCapture(video_file)

        # Check if the video file opened successfully
        if not cap.isOpened():
            print("Error: Could not open video file.")
            return
        
        # Get a unique video identifier from the file name
        video_id = os.path.splitext(os.path.basename(video_file))[0]
        self.output_images_path = os.path.join(self.output_images_path, video_id)
        
        # Initialize variables for frame saving
        if save_frames:
            os.makedirs(self.output_images_path, exist_ok=True)
            fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
            frame_interval = int(fps * frame_save_interval)  # Interval in frames
            saved_frame_count = 0
            frame_count = 0

        # Read and display frames in a loop
        while True:
            ret, frame = cap.read()  # Read a frame from the video
            if not ret:  # Break the loop if no frames are left
                print("End of video.")
                break

            # Display the current frame
            cv2.imshow('Video', frame)

            # Save frames if enabled
            if save_frames and frame_count % frame_interval == 0:
                output_path = os.path.join(self.output_images_path, f"{video_id}_frame_{saved_frame_count:04d}.jpg")
                cv2.imwrite(output_path, frame)
                saved_frame_count += 1

            # Increment frame counter
            if save_frames:
                frame_count += 1

            # Break the loop when 'q' is pressed
            if cv2.waitKey(25) & 0xFF == ord('q'):
                print("Video playback interrupted by user.")
                break

        # Release the video capture object and close all windows
        cap.release()
        cv2.destroyAllWindows()
        if save_frames:
            print(f"Saved {saved_frame_count} frames to '{self.output_images_path}'.")

    def load_data(self):
        # Load training data
        train_dataset = tf.keras.utils.image_dataset_from_directory(
            directory=self.output_images_path,
            image_size=(224, 224),  # Resize images
            batch_size=32,
            validation_split=0.2,  # 20% for testing
            subset="training",     # Load the training subset
            seed=123,              # Ensure reproducibility
            label_mode='int'
        )

        # Load testing data
        test_dataset = tf.keras.utils.image_dataset_from_directory(
            directory=self.output_images_path,
            image_size=(224, 224),
            batch_size=32,
            validation_split=0.2,  # 20% for testing
            subset="validation",   # Load the validation subset
            seed=123,
            label_mode='int'
        )

        # Display class names
        print(train_dataset.class_names)
        return train_dataset, test_dataset

In [63]:
guitarChordDetectModel = GuitarChordDetectModel(rf'C:\Users\user\Videos\Computer-Vision_Data\Guitar-Chords')

['C:\\Users\\user\\Videos\\Computer-Vision_Data\\Guitar-Chords\\A-Chord.mp4', 'C:\\Users\\user\\Videos\\Computer-Vision_Data\\Guitar-Chords\\Bm-Chord.mp4', 'C:\\Users\\user\\Videos\\Computer-Vision_Data\\Guitar-Chords\\C-Chord.mp4', 'C:\\Users\\user\\Videos\\Computer-Vision_Data\\Guitar-Chords\\D-Chord.mp4', 'C:\\Users\\user\\Videos\\Computer-Vision_Data\\Guitar-Chords\\G-Chord.mp4']
5 files


In [ ]:
guitarChordDetectModel.play_video_file(guitarChordDetectModel.file_names[0], True)

video file: C:\Users\user\Videos\Computer-Vision_Data\Guitar-Chords\G-Chord.mp4
End of video.
Saved 148 frames to 'data\frame_images\G-Chord'.


In [64]:
guitarChordDetectModel.output_images_path
train_dataset, test_dataset = guitarChordDetectModel.load_data()

Found 489 files belonging to 5 classes.
Using 392 files for training.
Found 489 files belonging to 5 classes.
Using 97 files for validation.
['A-Chord', 'Bm-Chord', 'C-Chord', 'D-Chord', 'G-Chord']


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>